---
# Chapter 8
## Object References, Mutability, and Recycling

---

## Variables are not boxes

---
### Example 8-1: Variables *a* and *b* hold references to the same list, not copies of the list

In [ ]:
a = [1, 2, 3]
b = a
a.append(4)
b

---
### Example 8-2: Variables are asigned to objects only after the objects are created

In [ ]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))


x = Gizmo()


In [ ]:
y = Gizmo() * 10

In [ ]:
dir()

## Identity, Equality, and Aliases

---
### Example 8-3: charles and lewis refer to the same object

In [ ]:
charles = {'name': 'Charles L. Dogson', 'born': 1832}
lewis = charles

In [ ]:
lewis is charles

In [ ]:
id(charles), id(lewis)

In [ ]:
lewis['balance'] = 950

In [ ]:
charles

---
### Example 8-4: alex and charles compare equal, but alex is not charles

In [ ]:
alex = {'name': 'Charles L. Dogson', 'born': 1832, 'balance': 950}
alex == charles

In [ ]:
alex is charles

## The relative immutability of Tuples

---
### Example 8-5: t1 and t2 initially compare equal, but changing a mutable item inside tuple t1 makes it different

In [ ]:
t1 = (1, 2, [20, 40])
t2 = (1, 2, [20, 40])

In [ ]:
t1 == t2

In [ ]:
id(t1[-1])

In [ ]:
t1[-1].append(99)
t1

In [ ]:
id(t1[-1])

In [ ]:
t1 == t2

## Copies Are Shallow by Default

---
### Example 8-6: Making a shallow copy of a list containng another list; copy and paste this code to see it animated at the Online Python Tutor ([https://pythontutor.com](https://pythontutor.com))

In [ ]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)      # 
l1.append(100)     # 
l1[1].remove(55)   # 
print('l1:', l1)
print('l2:', l2)
l2[1] += [33, 22]  # 
l2[2] += (10, 11)  # 
print('l1:', l1) 

## Deep and Shallow Copies of Arbitrary Objects

---
### Example 8-8: Bus pickl up and drops off passengers

In [ ]:
class Bus:

    def __init__(self, passangers=None):
        if passangers is None:
            self.passangers = []
        else:
            self.passangers = list(passangers)

    def pick(self, name):
        self.passangers.append(name)

    def drop(self, name):
        self.passangers.remove(name)


---
### Example 8-9: Effects of using copy versus deepcopy

In [ ]:
import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
print('id bus 1: {0}, id bus 2: {1}, id bus3: {2}'.format(id(bus1), id(bus2), id(bus3)))
bus1.drop('Bill')
print(bus2.passangers)
print('id passanger bus 1: {0}, id passanger bus 2: {1}, id passanger bus3: {2}'.format(id(bus1.passangers), id(bus2.passangers), id(bus3.passangers)))
print(bus3.passangers)

---
### Example 8-10: Cyclic referencesL b refers to a, and then is appended to a: deecopy still manages to copy a

In [ ]:
a = [10, 20]
b = [a, 30]
a.append(b)
print(a)

from copy import deepcopy
c = deepcopy(a)
print(c)

## Function Parameters as References

---
### Example 8-11: A function may change any mutable object it receives

In [ ]:
def f(a, b):
    a += b
    return a

x = 1
y = 2
f(x, y)

In [ ]:
x, y

In [ ]:
a = [1, 2]
b = [3, 4]
f(a, b)

In [ ]:
a, b

In [ ]:
t = (10, 20,)
u = (30, 40,)
f(t, u)

In [ ]:
t, u

## Mutable Types as Parameter Defaults: Bad Idea

---
### Example 8-12: A simple class to illustrate the danger of a mutable  default

In [ ]:
class HauntedBus:
    """A bus model haunted by ghot passengers"""
    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name: str) -> None:
        self.passengers.append(name)

    def drop(self, name: str) -> None:
        self.passengers.remove(name)

---
### Example 8-13: Buses haunted by ghost passengers

In [ ]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

In [ ]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

In [ ]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

In [ ]:
bus3 = HauntedBus()
bus3.passengers

In [ ]:
bus3.pick('Dave')
bus2.passengers

In [ ]:
bus2.passengers is bus3.passengers

In [ ]:
bus1.passengers

In [ ]:
dir(HauntedBus.__init__)

In [ ]:
HauntedBus.__init__.__defaults__

In [ ]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

## Defensive Programming with Mutable Parameters

---
### Example 8-14: Passengers disappear when dropped by a TwilightBus

In [ ]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

---
### Example  8-15: A simple class to show the perils of mutating received arguments

In [ ]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)


basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
basketball_team

## del and Garbage Collection

---
### Example 8-16: Watching the end of an object when no more references point to it

In [ ]:
import  weakref

s1 = {1, 2, 3}
s2 = s1

def bye():
    print('Gone with the wind...')

ender = weakref.finalize(s1, bye)
print(ender.alive)
del s1
print(ender.alive)
s2 = "spam"
print(ender.alive)

## Weak References

---
### Example 8-17: A weak reference is a callable that returns the references object or None if the referencet is not more

In [ ]:
import weakref

a_set = {0, 1}
wref = weakref.ref(a_set)
wref

In [ ]:
wref()

In [ ]:
a_set = {2, 3, 4}
wref()

In [ ]:
wref() is None

In [ ]:
wref() is None

## The WeakValueDictionary Skit

---
### Example 8-18:  Cheese has a kind attribute and standard representation

In [ ]:
class Cheese:

    def __init__(self, kind):
        self.kind = kind

    def __repr__(self):
        return 'Cheese(%r)' % self.kind

---
### Example 8-19: Customer: Have you in fact got any cheese here at all?"

In [ ]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
    stock[cheese.kind] = cheese

print(sorted(stock.keys()))

del catalog
del cheese
print(sorted(stock.keys()))

## Limitations of Weak References

In [ ]:
class MyList(list):
    """asdasd"""
    pass

a_list  = MyList(range(10))

wref_to_a_list = weakref.ref(a_list)

## Trics Python Plays with Immutables

### Example 8-20: A tuple built from another is actually the same exact tuple

In [ ]:
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t2

In [ ]:
t3 = t1[:]
t3 is t1

### Example 8-21: String literals may create shared objects

In [ ]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1

In [ ]:
s1 = "ABC"
s2 = "ABC"
s2 is s1